<a href="https://colab.research.google.com/github/thiagonobrega/tl-pprl/blob/master/notebooks/competitors/TransferER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning TransER repo

In [ ]:
!git clone https://github.com/nishadi/TransER
!mv TransER/data .

fatal: destination path 'TransER' already exists and is not an empty directory.
mv: cannot stat 'TransER/data': No such file or directory


instaling libs and creating folder

In [ ]:
!rm -rf datasets
!mkdir datasets

!pip install --user gdown==4.4.0  &> /dev/null
!pip install tqdm &> /dev/null

In [ ]:
import os
import sys
import pickle
import zipfile
import logging

import numpy as np
import pandas as pd

from tqdm.notebook import trange, tqdm

sys.path.append('TransER/')
from transer import model

## get datasets

In [ ]:
!rm -rf datasets
!mkdir datasets

personal_ds_links = [
                     '1J-4xHTanKik7Nytm5T_Y7yYiBqi3Ej9a', #yv-er
                     '1ugVFwn93bAQrxp5p7hICJRAv2igqUXNH', #tse
                     '1TWT-Veh786NVal7Rz8VK1xzaJWmv6Nsd', #ncvr
                     '1V9SLWLFjsySHYTjpzMcMlM23atQTPc9D', #mvr
                     '1iKlLBiAJq7UvT3woOcV-zrQHKN6QKubA', #census
]


for id in tqdm(personal_ds_links,leave=False):
  !gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
file_list = [ 
            #  ('abt-buy','https://storage.live.com/downloadfiles/V1/Zip?authKey=!ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349104&canary=&authkey=%21ANJgdUEuUvK8MX0'),
             ('books','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349105&canary=&authkey=%21ANJgdUEuUvK8MX0'),
             ('movies','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349108&canary=&authkey=%21ANJgdUEuUvK8MX0'),
             ('restaurants','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349111&canary=&authkey=%21ANJgdUEuUvK8MX0'),
]

for fi in tqdm(file_list):
  context=fi[0]
  url=fi[1]
  param=fi[2]
  !wget -N -O {context}.zip --post-data='{param}' '{url}' 
  !unzip -o {context}.zip -d datasets/{context} 
  

  0%|          | 0/3 [00:00<?, ?it/s]

for details.

--2022-05-30 19:36:38--  https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648
Resolving storage.live.com (storage.live.com)... 40.90.130.192
Connecting to storage.live.com (storage.live.com)|40.90.130.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘books.zip’

books.zip               [            <=>     ]  39.68M  2.86MB/s    in 14s     

2022-05-30 19:36:53 (2.88 MB/s) - ‘books.zip’ saved [41607538]

Archive:  books.zip
 extracting: datasets/books/comparison_books_amazon-barnesnobel.zip  
 extracting: datasets/books/comparison_books_amazon-barnesnobel_blk_1.zip  
 extracting: datasets/books/comparison_books_amazon-barnesnobel_blk_2.zip  
 extracting: datasets/books/comparison_books_amazon-barnesnobel_blk_3.zip  
 extracting: datasets/books/comparison_books_amazon-barnesnobel-small.zip  
 extracting: datasets/books/comparison_books_amazon-barnesnobel-small_b

## metodos

In [ ]:
def open_ds(infile , n_atts='atts-1',blk=None):
    '''

    Open the comparisons files

    :param infile:
    :param n_atts:
    :param deduplica:
    :return:
    '''
    zf = zipfile.ZipFile(infile)
    nl = zipfile.ZipFile.namelist(zf)

    filtering_key = n_atts
    if blk != None:
        filtering_key = n_atts + '-' +str(blk) + '.csv'
    
    for i in range(0, len(nl)):
        fn = nl[i]

        if (filtering_key in fn):
            a = pd.read_csv(zf.open(fn), header=0, sep=";",
                            index_col=False)

            # remove comparacoes com mesma id
            # colocar flag
            # if (deduplica):
            #     a = a[a['id1'] != a['id2']]

    return a

In [ ]:
def convert_data(context,csv_file,atts,
                 blk=None,
                 ds_dir='datasets/',exp_dir='data/'):
  
  s_file = ds_dir + context + csv_file
  # path = exp_dir + context.upper()
  path = exp_dir + context
  os.makedirs(path, exist_ok = True)
  
  # print(range(len(atts)))
  for i in trange(len(atts),leave=False):
    att = atts[i]
    df = open_ds(s_file,n_atts='atts-'+str(att))

    df_dict = {}
    # df_dict['features'] = df.iloc[:,2:7]
    df_dict['features'] = df.iloc[:,2:6]
    df_dict['labels']   = df.iloc[:,-1:].squeeze() # to series

    with open(path + '/sampled-A-A-'+str(i)+'.pkl', 'wb') as handle:
      pickle.dump(df_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return df_dict

In [ ]:
def write_results_2_google(df, google_auth,
                  spreadsheetId='invalid_id',
                  sheetName = 'Sheet1'):
  
  """
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds) #google_auth
  """

  sh = google_auth.open_by_key(spreadsheetId)
  sheet = sh.worksheet(sheetName)

  values = df.values.tolist()
  #append_row
  sh.values_append(sheetName, {'valueInputOption': 'USER_ENTERED'}, {'values': values})

## setup

In [ ]:
###
### Source
###

ds_source_all_blk = [
            ("books",['comparison_books_amazon-barnesnobel_blk_1.zip','comparison_books_amazon-barnesnobel-small_blk_1.zip'],3,None),
            ("books",['comparison_books_goodreads-barnesnobel_blk_1.zip'],4,None),
            ("movies",['comparison_movies_imdb-rottentomatos_blk_1.zip'],3,None),
            ("movies",['comparison_movies_imdb-tmd_blk_1.zip'],2,None),
            ("restaurants",['comparison_restaurants_fodors-zagats_blk_1.zip'],4,None),
          ]
###
### Target 
###

#context, file list, atts , blk
ds_target_blk = [                 
            ("ncvr",['comparison_ncvoters_10000_0.1_blk.zip'],3,2),
            ("mvr",['comparison_michiganvoters_10000_0.1_blk.zip'],3,2),
            ("tse",['comparison_tse_2000_0.1_blk_1.zip'],3,None), #blk?4 $ PROBLEMA
            ("census",['comparison_census_blk.zip'],4,2), #blk-2
            # ("yv-er",['comparison_processed_yver_blk.zip'],4,2),#blk-2
            ("yv-er",['comparison_yver_full.zip'],4,None)#blk-2
            
          ]

In [ ]:
scenario = 'transER'
# scenario = '2'
ds_source = ds_target_blk + ds_source_all_blk

ds_target = ds_target_blk
# ds_target = ds_target_full

## run

### converting data

In [ ]:
for i in tqdm(ds_source,leave=False):
  folder = i[0] + os.sep
  arquivo  = i[1][0]
  att      = i[2]
  blk      = i[3]

  # print(folder,arquivo,att,blk)
  !rm -rf data/{folder}
  if (folder == 'tse/') or (folder == 'census/'):
    z = convert_data(folder,arquivo,[str(att),str(att),str(att)],blk=blk)
  elif (folder == 'yv-er/'):
    z = convert_data(folder,arquivo,[str(att),str(att-2),str(att-1)],blk=blk)
  elif (folder == 'books/') or (folder == 'movies/') or (folder == 'restaurants/'):
    z = convert_data(folder,arquivo,[str(att),str(att),str(att)],blk=blk)
  elif att > 2:
    # z = convert_data(folder,arquivo,[str(att),str(att-2),str(att-1)],blk=blk)
    z = convert_data(folder,arquivo,[str(att),str(att-2),str(att+1)],blk=blk)
  else:
    z = convert_data(folder,arquivo,[str(att),str(att-1),str(att+1)],blk=blk)

print('All done')
# !rm -rf data/ncvr
# !rm -rf data/mvr
# a = convert_data('mvr/','comparison_michiganvoters_10000_0.1_blk.zip', ['1','3','4'])
# b = convert_data('ncvr/','comparison_ncvoters_10000_0.1_blk.zip', ['1','3','4'])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

All done


### run

In [ ]:
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)
!rm transER_exec.log

# formatter = logging.Formatter('%(asctime)s %(levelname)s: %(funcName)s:%(lineno)d %(message)s')
formatter = logging.Formatter('%(funcName)s:%(lineno)d %(message)s')

# create logger with 'spam_application'
logger = logging.getLogger()
logger.handlers.pop(0)
# logger.handlers

logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('transER_exec.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)

# just so we can see things as they happen in stdout/stderr
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)


In [ ]:
def run_transER(src_dataset, src_link, tgt_dataset, tgt_link, k=7, t_c=0.9,
               t_l=0.9, t_p=0.99, samples=1):

  result_list = list()
  final_result = []
  # Iterate through different blocking samples

  for i in range(samples):
    # Retrieve source dataset
    src_dataset_file = 'data/{}/sampled-{}-{}.pkl'.format(src_dataset,
                                                          src_link, i)
    src_feature_dict = pickle.load(open(src_dataset_file, 'rb'))
    src_fts, src_labels = src_feature_dict['features'], src_feature_dict[
      'labels']

    # Retrieve target dataset
    tgt_dataset_file = 'data/{}/sampled-{}-{}.pkl'.format(tgt_dataset,
                                                          tgt_link, i)
    tgt_feature_dict = pickle.load(open(tgt_dataset_file, 'rb'))
    tgt_fts, tgt_labels = tgt_feature_dict['features'], tgt_feature_dict[
      'labels']

    # Match data with TransER
    # retorno = [src_dataset,tgt_dataset] #saida
    retorno = { 'source' : src_dataset, 'target': tgt_dataset}
    try:
      tmp_result_list = model.match_data(src_fts, src_labels, tgt_fts, tgt_labels, k,
                                 t_c, t_l, t_p)
      result_list.extend(tmp_result_list)
      # save results
      #precision
      p_list = [float(r['precision']) for r in result_list]
      r_list = [float(r['recall']) for r in result_list]
      f1_list = [float(r['f']) for r in result_list]
      meta_list = [str(r['meta']) for r in result_list]

      tamanho = np.max([len(p_list),len(r_list),len(f1_list),len(meta_list)])

      
      for i in range(0,tamanho):
        retorno = { 'source' : src_dataset, 'target': tgt_dataset}
        retorno['alg'] = meta_list[i]
        retorno['precision'] = p_list[i]
        retorno['recall'] = r_list[i]
        retorno['f1'] = f1_list[i]
        final_result.append(retorno)
        # retorno.append(meta_list[i]) #precision
        # retorno.append(p_list[i]) #precision
        # retorno.append(r_list[i]) #recall
        # retorno.append(f1_list[i]) #f1

    except ValueError:
      logging.debug(">>>> Unable to execute :: " + tgt_dataset +" - "+ src_dataset)
      retorno = { 'source' : src_dataset, 'target': tgt_dataset}
      retorno['alg'] = 'none'
      retorno['precision'] = -1
      retorno['recall'] = -1
      retorno['f1'] = -1
      final_result.append(retorno)
      

  return final_result

In [ ]:
saida = []

for tgt in tqdm(ds_target,leave=False):
  tgt_dataset = tgt[0]#'NCVR'  
  tgt_link = 'A-A'
  for src in tqdm(ds_source,leave=False):
    src_dataset = src[0]#'MVR'
    src_link = 'A-A'
    if tgt_dataset != src_dataset:
      # try:
      logging.debug("Executing :: target=" + tgt_dataset +", source="+ src_dataset)
        # model.predict(src_dataset, src_link, tgt_dataset, tgt_link)
      for r in run_transER(src_dataset, src_link, tgt_dataset, tgt_link, samples=1):
        saida.append(r)
      # except ValueError:
      #   logging.debug(">>>> Unable to execute :: " + tgt_dataset +" - "+ src_dataset)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
df = pd.DataFrame(saida)

In [ ]:
#autentica no google
from google.colab import auth
import gspread
from google.auth import default


auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
write_results_2_google(df,gc,
              spreadsheetId='1AhdK5O6oZFHZSXLsBPI59YDj9GSF8AGJzNTnCi12hqM',
              sheetName='qualidade_'+scenario)

## scratch